In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from helpers import data_handler as dh

# Background

Ideas:

1. Give users who have older accounts more weight in calculating the optimal timestamps
2. Give users who used more words in the twitch chat more weight

## User Stats

In [2]:
def user_stats(dataframe):
    '''
    Returns the user who sent the most messages, most emoticons, and who changed their username the most
    '''
    temp_df = dataframe.groupby("name").count().reset_index()
    temp_df['username_chg'] = temp_df['_id'] - temp_df['display_name'] # if mismatch in count, then its probs changed username
    
    stats = {}
    for col in ['_id','emoticons', 'username_chg']:
        sort = temp_df.sort_values(col, ascending=False)[['name',col]].iloc[0]
        stat = sort[col]
        _id = sort['name']
        
        if col == '_id':
            col = 'num_messages'
        stats[col] = [_id, stat]
    
    return stats

# Find number of words for each user

In [3]:
data = json.load(open('data/big_data.json'))
big_df = dh.organize_twitch_chat(data)
big_df = big_df.astype({'_id':int})

In [4]:
id_words = pd.DataFrame(columns = ['display_name','_id','num_words', 'num_emoji'])
for _id in big_df['_id'].unique():
    temp_df = big_df[big_df['_id'] == _id]
    words = temp_df['body'].str.split(' ')
    emoji = temp_df['emoticons'].apply(lambda x: 0 if type(x) == float else len(x))
    num_emoji = emoji.sum()
    num_words = words.apply(lambda x: len(x))
    
    sum_words = num_words.sum()
    id_words = id_words.append({
        'display_name':temp_df['display_name'].iloc[0],
        '_id':_id,
        'num_words':sum_words,
        'num_emoji':num_emoji
    }, ignore_index=True)

In [5]:
id_words = id_words.astype({'_id':int,'num_words':int,'num_emoji':int})
id_words['only_words'] = id_words['num_words'] - id_words['num_emoji']

In [6]:
id_words.corr()

,_id,num_words,num_emoji,only_words
_id,1.000000,0.088643,0.078218,0.088360
num_words,0.088643,1.000000,0.770062,0.999850
num_emoji,0.078218,0.770062,1.000000,0.758908
only_words,0.088360,0.999850,0.758908,1.000000


In [7]:
id_words = id_words.sort_values('only_words',ascending=False).reset_index(drop=True).reset_index()
id_words = id_words.rename({'index':'rank'},axis=1)

In [8]:
id_words.head(10)

,rank,display_name,_id,num_words,num_emoji,only_words
0,0,DominickStarcraft,27004161,1343,25,1318
1,1,StreamElements,100135110,919,1,918
2,2,kimpossible489,26298161,765,18,747
3,3,ninaisnoob,488814751,765,20,745
4,4,gassyvegan247,495717828,673,27,646
5,5,PeekGame,125488559,469,1,468
6,6,wall_lii,637481218,469,15,454
7,7,Stryg_HS,193624801,419,19,400
8,8,kubetz,21584081,360,10,350
9,9,coookiemonst,75347949,365,19,346


## Create user weight

In [9]:
# Weight = "num words / num words of biggest spammer"
id_words['weight'] = id_words['only_words'] / id_words['only_words'].max()

# Split dataframe

In [11]:
first_stamp, chunk_list = dh.get_chunks(big_df)

/home/jupyter-pomkos/projects/twitch_chat_analysis/helpers/data_handler.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['hour'] = i
/home/jupyter-pomkos/projects/twitch_chat_analysis/helpers/data_handler.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['chunk'] = x


In [12]:
user_dict = {} # store chunks of the top 10 users
for user in id_words.head(10)['_id']:
    user_dict[user] = []

for user in user_dict.keys():
    for chunk in chunk_list:
        if sum(chunk['_id'] == user) > 5:
            user_dict[user].append(chunk)

In [13]:
top_chunks = []
for user, value in user_dict.items():
    if len(value) > 0:
        for chunk in value:
            chunk['num_top_user_appears'] = chunk['_id']==user
            top_chunks.append(chunk)

`top_dict` now contains all chunks where at least one of the top 10 users appears more than 5 times

In [14]:
def results_formatter(list_chunk, goal):
    '''
    Creates a new df `results` that contains the total number of words in the dataframe, the time
    the time the dataframe started and ended
    input
    -----
    list_chunk: list
        List of pd.DataFrame
    goal: str
        Col name that has calculated results (ex: num_words, chat_rate, etc.)
    output
    ------
    results: pd.DataFrame
        Dataframe with `start`, `end`, `num_words` columns
    '''
    chunk_list = []
    time_start_list = []
    time_end_list = []
    goal_list = []
    
    for chunk in list_chunk:
        time_start_list.append(chunk.iloc[0, 0])  # assume created_at col is first
        time_end_list.append(chunk.iloc[-1, 0])  # assume created_at col is first
        goal_list.append(chunk[goal].sum())

    results = pd.DataFrame({
        'start':time_start_list,
        'end':time_end_list,
        goal:goal_list
    })
    results = results.sort_values(goal, ascending=False).drop_duplicates()
    return results

In [15]:
results = results_formatter(top_chunks, goal='num_top_user_appears')

In [16]:
json_results = dh.results_jsonified(results, first_stamp, results_col = 'num_top_user_appears')
dh.save_json(json_results,'algo3_top_user_appears')

Saved to data/algo3_top_user_appears.json


In [43]:
########
# This script finds the number of words/emojis/both depending on `goal` variable,
# isolates to each `min_` timestamp, and sorts the resulting df by largest number
########
import pandas as pd
from helpers import data_handler as dh

def thalamus(big_df, min_, goal, min_words):
    
    id_words = id_words_counter(big_df)
    first_stamp, chunk_list = dh.get_chunks(big_df)
    top_chunks = new_chunk_list(id_words, chunk_list, min_words = 5)
    results = results_formatter(top_chunks, goal='num_top_user_appears')
    
    return results, first_stamp
    
def results_formatter(list_chunk, goal):
    '''
    Creates a new df `results` that contains the total number of words in the dataframe, the time
    the time the dataframe started and ended
    input
    -----
    list_chunk: list
        List of pd.DataFrame
    goal: str
        Col name that has calculated results (ex: num_words, chat_rate, etc.)
    output
    ------
    results: pd.DataFrame
        Dataframe with `start`, `end`, `num_words` columns
    '''
    chunk_list = []
    time_start_list = []
    time_end_list = []
    goal_list = []
    
    for chunk in list_chunk:
        time_start_list.append(chunk.iloc[0, 0])  # assume created_at col is first
        time_end_list.append(chunk.iloc[-1, 0])  # assume created_at col is first
        goal_list.append(chunk[goal].sum())

    results = pd.DataFrame({
        'start':time_start_list,
        'end':time_end_list,
        goal:goal_list
    })
    results = results.sort_values(goal, ascending=False).drop_duplicates()
    return results
    
def new_chunk_list(id_words, chunk_list, min_words):
    '''
    Creates a new list of chunks, containing only chunks where top
    users sent more than `min_words` words
    '''
    user_dict = {} # store chunks of the top 10 users
    for user in id_words.head(10)['_id']:
        user_dict[user] = []
    for user in user_dict.keys():
        for chunk in chunk_list:
            chunk['_id'] = chunk['_id'].astype(int)
            if sum(chunk['_id'] == user) > min_words:
                user_dict[user].append(chunk)
    top_chunks = []
    for user, value in user_dict.items():
        if len(value) > 0:
            for chunk in value:
                chunk['num_top_user_appears'] = chunk['_id']==user
                top_chunks.append(chunk)
    return top_chunks

def id_words_counter(big_df):
    '''
    Returns a dataframe with all user IDs and the number of words/emojis/combined
    they each sent, sorted by top senders
    '''
    id_words = pd.DataFrame(columns = ['display_name','_id','num_words', 'num_emoji'])
    
    for _id in big_df['_id'].unique():
        temp_df = big_df[big_df['_id'] == _id]
        words = temp_df['body'].str.split(' ')
        emoji = temp_df['emoticons'].apply(lambda x: 0 if type(x) == float else len(x))
        num_emoji = emoji.sum()
        num_words = words.apply(lambda x: len(x))

        sum_words = num_words.sum()
        id_words = id_words.append({
            'display_name':temp_df['display_name'].iloc[0],
            '_id':_id,
            'num_words':sum_words,
            'num_emoji':num_emoji
        }, ignore_index=True)
    id_words = id_words.astype({'_id':int,'num_words':int,'num_emoji':int})
    id_words['only_words'] = id_words['num_words'] - id_words['num_emoji']
    id_words = id_words.sort_values('only_words',ascending=False).reset_index(drop=True).reset_index()
    id_words = id_words.rename({'index':'rank'},axis=1)
    # Weight = "num words / num words of biggest spammer"
    id_words['weight'] = id_words['only_words'] / id_words['only_words'].max()
    
    return id_words
    
def run(data, min_, min_words):
    data = pd.DataFrame.from_records(data)
    big_df = dh.organize_twitch_chat(data) # fetch appropriate data
    results, first_stamp = thalamus(big_df, min_, min_words = 5, goal='num_top_user_appears') 
    
    json_results = dh.results_jsonified(results, first_stamp, results_col='num_top_user_appears')
    dh.save_json(json_results, f"algo3.0_top_user_appears")
    
    return json_results, results

In [44]:
json_results, results = run(data,2,2)

/home/jupyter-pomkos/projects/twitch_chat_analysis/helpers/data_handler.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['hour'] = i
/home/jupyter-pomkos/projects/twitch_chat_analysis/helpers/data_handler.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['chunk'] = x


Saved to data/algo3.0_top_user_appears.json
